In [ ]:
# %%bash
# pip install "ray[client]"==2.9.0

In [1]:
import sys
import time
from collections import Counter

import ray

In [2]:
@ray.remote(num_cpus=1)
def gethostname(x):
    import platform
    import time

    time.sleep(0.01)
    return x + (platform.node(),)


def wait_for_nodes(expected):
    # Wait for all nodes to join the cluster.
    while True:
        resources = ray.cluster_resources()
        node_keys = [key for key in resources if "node" in key]
        num_nodes = sum(resources[node_key] for node_key in node_keys)
        if num_nodes < expected:
            print(
                "{} nodes have joined so far, waiting for {} more.".format(
                    num_nodes, expected - num_nodes
                )
            )
            sys.stdout.flush()
            time.sleep(1)
        else:
            break


def main():
    wait_for_nodes(2)

    # Check that objects can be transferred from each node to each other node.
    for i in range(10):
        print("Iteration {}".format(i))
        results = [gethostname.remote(gethostname.remote(())) for _ in range(100)]
        print(Counter(ray.get(results)))
        sys.stdout.flush()

    print("Success!")
    sys.stdout.flush()

In [4]:
ray.init(address="ray://raycluster-kuberay-head-svc.default.svc.cluster.local:10001")

Python version:,3.8.13
Ray version:,2.2.0
Dashboard:,http://10.244.0.65:8265


In [5]:
main()

Iteration 0
Counter({('raycluster-kuberay-worker-workergroup-rqvh7', 'raycluster-kuberay-head-862db'): 37, ('raycluster-kuberay-head-862db', 'raycluster-kuberay-worker-workergroup-rqvh7'): 27, ('raycluster-kuberay-worker-workergroup-rqvh7', 'raycluster-kuberay-worker-workergroup-rqvh7'): 18, ('raycluster-kuberay-head-862db', 'raycluster-kuberay-head-862db'): 18})
Iteration 1
Counter({('raycluster-kuberay-head-862db', 'raycluster-kuberay-head-862db'): 34, ('raycluster-kuberay-worker-workergroup-rqvh7', 'raycluster-kuberay-head-862db'): 24, ('raycluster-kuberay-head-862db', 'raycluster-kuberay-worker-workergroup-rqvh7'): 22, ('raycluster-kuberay-worker-workergroup-rqvh7', 'raycluster-kuberay-worker-workergroup-rqvh7'): 20})
Iteration 2
Counter({('raycluster-kuberay-head-862db', 'raycluster-kuberay-head-862db'): 32, ('raycluster-kuberay-worker-workergroup-rqvh7', 'raycluster-kuberay-head-862db'): 25, ('raycluster-kuberay-worker-workergroup-rqvh7', 'raycluster-kuberay-worker-workergroup-rq

### Access Ray dashboard

Run a port-forward from your local computer to the ray dashboard:

`kubectl port-forward svc/raycluster-kuberay-head-svc 8265:8265 -n default`

Go to http://localhost:8265/ and you should see the Ray dashboard.

### Connect to Ray cluster from local machine

> The version of the Ray client must match the version of the Ray cluster (2.9.0).

```bash
conda create -n ray-env python=3.8.10
conda activate ray-env
pip install "ray[client]"==2.9.0
```

Run a port forwarding to the Ray head service:

```bash
kubectl port-forward svc/raycluster-kuberay-head-svc 10001:10001 -n default
```

```python
import ray

ray.init(address="ray://localhost:10001")
print(ray.cluster_resources())
```